In [188]:
import numpy as np
import pandas as pd
import re,math
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
data1 = pd.read_csv(r'BCHAIN-MKPRU.csv') # B
data2 = pd.read_csv(r'LBMA-GOLD.csv') # G

In [189]:
# 线性插值填充
data2.interpolate(method='linear', limit_direction='backward', axis=0, inplace = True)
zdata = pd.merge(data1, data2, how='outer').fillna(method='ffill').fillna(method='backfill')

In [190]:
def amplitude(list_1): # 涨幅跌幅计算函数
    x1 = list_1.copy()[1:]
    x2 = list_1.copy()[:-1]
    y = (x1 - x2)/x2
    return y

In [191]:
# 比特币日涨幅
day_b_amp = amplitude(np.array(data1["Value"]))
# 黄金日涨幅
day_amp = amplitude(np.array(data2["USD (PM)"]))

In [192]:
def Rt(Pt):
    p1 = np.array(Pt.copy()[1:])
    p2 = np.array(Pt.copy()[:-1])
    Rt = (p1 - p2)/p2
    return Rt

def ERt(Rt):
    ERt = []
    n = len(Rt)
    for i in range(1,n+1):
        ERt.append(np.array(list(Rt[0:i])).sum()/i)
    return ERt

def Dt(Rt):
    DRt = []
    n = len(Rt)
    for i in range(1,n+1):
#         tpd = 0
#         for j in range(1,i):
#             tpd = tpd+(Rt[j]-ERt(Rt[0:i])[-1])**2
#         DRt.append(tpd/(i-1))
        DRt.append(np.std(Rt[0:i],ddof = 1))
    return DRt

Rgt = Rt(zdata["USD (PM)"])
ERgt = ERt(Rgt)
DRgt = Dt(Rgt)
Rbt = Rt(zdata["Value"])
ERbt = ERt(Rbt)
DRbt = Dt(Rbt)

D:\Anaconda\envs\python32\lib\site-packages\numpy\core\_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
D:\Anaconda\envs\python32\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [193]:
def gold(short_item, long_item, data, day_amp, status_dict):
    #判断是否在第一个中周期内
    if len(data) <= short_item:
        return None

    # 周黄金平均值
    count = 1
    temp3 = []
    while True:
        tp = data.iloc[short_item*(count-1): short_item*count]["USD (PM)"].mean()
        count+= 1
        temp3.append(tp)
        if count >= len(data)/short_item:
            break
    temp3_df = pd.DataFrame(temp3) # 252周的金价
    # 长期(20天)
    count = 1
    temp4_max = [] # 周高位点
    temp4_min = [] # 周低位点
    while True:
        tp_max = max(temp3[(count-1)*short_item:count*short_item])
        tp_min = min(temp3[(count-1)*short_item:count*short_item])
        temp4_max.append(tp_max)
        temp4_min.append(tp_min)
        count+=1
        if count >= len(temp3)/short_item:
            break
    temp4_max_h = amplitude(np.array(temp4_max)) # 周高位点
    temp4_min_l = amplitude(np.array(temp4_min)) # 周低位点
    #周趋势(同增同减同趋势，不同则未知)
    tptp = []
    for i,j in zip(temp4_max_h, temp4_min_l):
        if i > 0 and j > 0:
            tptp.append(1)
        elif i < 0 and j < 0:
            tptp.append(-1)
        else:
            tptp.append(0)
    tptp_w_g = pd.DataFrame(tptp) # 第二周开始的61周涨跌幅度表 -1跌+1涨


    # 5天(一周趋势)
    count = 1
    temp5_max = []
    temp5_min = []
    while True:
        tp_max = max(np.array(data["USD (PM)"])[(count-1)*short_item:count*short_item])
        tp_min = min(np.array(data["USD (PM)"])[(count-1)*short_item:count*short_item])
        temp5_max.append(tp_max)
        temp5_min.append(tp_min)
        count+=1
        if count >= len(data)/short_item:
            break
    temp5_max_h = amplitude(np.array(temp5_max))
    temp5_min_l = amplitude(np.array(temp5_min))
    #
    tptp = []
    count = 0
    for i,j in zip(temp5_max_h, temp5_min_l):
        if i > 0 and j > 0:
            tptp.append(1)
        elif i < 0 and j < 0:
            tptp.append(-1)
        else:
            tptp.append(0)
        count += 1
    tptp_5 = pd.DataFrame(tptp)
    

    #判断是否在第一个长周期内
    if len(data) < short_item*long_item+1: 
        print(tptp_w_g)
        # 第一个长周期内
        # 买入-1 卖出+1
        for i in range(long_item):
            if i == 0: # 第二周
                print(tptp_w_g)
                status_first_5 = np.array(tptp_w_g)[0][0]
                if status_first_5 < 0:
                    status = -1 # 跌
                elif status_first_5 > 0:
                    status = 1 # 涨
                else:
                    status = 0 
            else: # 第三周以后20天以内
                for j in range(short_item):
                    day = short_item*i+j+short_item-1
                    if day == 19:
                        print(status)
                    if day >= len(data):
                        if status_dict == []: # 没有任何买入时
                            if status == 1:
                                status = np.array(tptp_w_g)[i-1][0]
                                continue
                            elif status == -1 and day_amp[day] > 0:
                                status_dict.append((day, -1, np.array(data)[day][1])) # 买入
                                status = np.array(tptp_w_g)[i-1][0]
                                print(status_dict)
                                break
                        else: # 有买入后
                            #  上周涨跌判断      本周每日涨跌幅        上笔交易买卖情况   
                            if status == -1 and day_amp[day] > 0 and status_dict[-1][1] >= 0: # 上一笔是卖出才能买入
                                status_dict.append((day, -1, np.array(data)[day][1])) # 买入
                                status = np.array(tptp_w_g)[i-1][0]
                                print(status_dict)
                                break
                            elif status == -1 and day_amp[day] < 0 and status_dict[-1][1] <= 0: # 上一笔是买入才能卖出
                                status_dict.append((day, 1, np.array(data)[day][1])) # 卖出
                                status = np.array(tptp_w_g)[i-1][0]
                                print(status_dict)
                                break
                    else:
                        status = np.array(tptp_w_g)[i-1][0]
    else:#不在第一个长周期内
        # 第二个月以及后面
        for k in range(len(tptp_w_g)):
            if np.array(tptp_w_g)[k][0] == 0:
                status = np.array(tptp_w_g)[k-1][0]
            else:
                status = np.array(tptp_w_g)[k][0]
            for i in range(long_item):
                for j in range(short_item):
                    day = long_item*short_item*(k+1)+short_item*i+j
                    if day >= len(data):
                        if status_dict == []: # 前期无交易
                            if status == -1 and day_amp[day] > 0: # 上一笔是卖出才能买入或无上一笔可买入
                                status_dict.append((day, -1, np.array(data)[day][1])) # 买入
                                status = np.array(tptp_w_g)[i-1][0]
                                print(status_dict)
                                break
                            elif status == -1 and day_amp[day] < 0: # 上一笔是买入才能卖出，无上一笔的不能卖
                                status = np.array(tptp_w_g)[i-1][0]
                                continue
                        else: # 前期有交易
                            # 跌幅超过10%交易避免赔本
                            if status == -1 and ((np.array(data)[day][1]-status_dict[-1][-1])/status_dict[-1][-1]) <= -0.1 and status_dict[-1][1] <= 0:
                                status_dict.append((day, -1, np.array(data)[day][1])) # 卖出
                                print(status_dict)
                                break
                            if status == -1 and day_amp[day] > 0 and status_dict[-1][1] >= 0: # 上一笔是卖出才能买入
                                status_dict.append((day, -1, np.array(data)[day][1])) # 买入
                                status = np.array(tptp_w_g)[i-1][0]
                                print(status_dict)
                                break
                            elif status == -1 and day_amp[day] < 0 and status_dict[-1][1] <= 0: # 上一笔是买入才能卖出
                                status_dict.append((day, 1, np.array(data)[day][1])) # 卖出
                                status = np.array(tptp_w_g)[i-1][0]
                                print(status_dict)
                                break
                    else:
                        status = np.array(tptp_w_g)[i-1][0]

    # 保存 交易日期 交易情况 交易当日金价
    temp6 = []
    temp6_in_index = []
    temp6_in_value = []
    temp6_out_index = []
    temp6_out_value = []
    for i in status_dict:
        temp6.append([i[0], i[1], np.array(data)[i[0]][1]])
        if i[1] == -1:
            temp6_in_index.append(i[0])
            temp6_in_value.append(i[2])
        elif i[1] == 1:
            temp6_out_index.append(i[0])
            temp6_out_value.append(i[2])
    output1 = pd.DataFrame(temp6, columns=["Date", "process", "price"])

    if status_dict != []:
        if status_dict[0][0] == len(data):
            return status_dict,True
        else:
            return status_dict,False
    else:
        return status_dict,False

In [194]:
data2[0:10]

,Date,USD (PM)
0,9/12/16,1324.60
1,9/13/16,1323.65
2,9/14/16,1321.75
3,9/15/16,1310.80
4,9/16/16,1308.35
5,9/19/16,1314.85
6,9/20/16,1313.80
7,9/21/16,1326.10
8,9/22/16,1339.10
9,9/23/16,1338.65


In [195]:
short_item = 9
long_item = 2
status_dict = []
for i in range(19,len(data2)):
    data = data2.iloc[0:i]
    temp = gold(short_item, long_item, data, day_amp[0:i], status_dict)
    status_dict = temp[0]
    if temp[1] == True:
        print(c)
    else:
        pass